In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import os
import subprocess
import json

from collections import defaultdict, Counter
from tqdm import tqdm_notebook

%matplotlib inline

In [8]:
class OnlineLogisticRegression():
    
    """ OnlineLogisticRegression classifier
    Класс реализует модель multiclass/multilabel классификации текстов, используя методы онлайн 
    обучения (стохастический градиентный спуск). В качестве регуляризатора используется ElasticNet 
    (комбинация L1 и L2). Поддерживаются повторные проходы по тренировочному датасету и ограничения 
    на размер словаря.
    
    Parameters
    ----------
    tags : [string]
        Список допустимых для классификации тегов (классов). Не входящие в этот список теги 
        игнорируются.

    strategy : ['ovr', 'multinomial'], default: 'ovr'
        Признак типа классификации. Значение 'ovr' задает бинарную классификацию (присутствие/отсутствие) 
        для каждого тега, теги независимы (one-vs-rest, multilabel classification). Значение 
        'multinomial' задает минимизацию ошибки общего дискретного вероятностного распределения 
        (multiclass classification).
    
    learning_rate : float, default: 0.1
        Скорость обучения градиентного спуска (множитель корректировки параметра модели на каждом шаге).

    lmbda : float, default: 0.0002
        Коэффициент ElasticNet-регуляризации на каждом шаге.

    gamma : float, default: 0.1
        Вес L2-компоненты в ElasticNet.
        
    store_frequency : bool, default: True
        Флаг хранения частот слов в выборке для последующего снижения размерности признакового 
        пространства (сильно понижающая скорость первоначального обучения операция).

    tolerance : float, default: 1e-16
        Порог для огнаничения значений аргумента логарифма.
    
    Attributes
    ----------
    vocab_ : dict {string: int}
        Mapping слов-признаков в численные индексы. Слова добавляются в словарь в процессе обучения, 
        индексы назначаются инкрементально.
        Т.к. обучение модели ведется по онлайн-схеме, то всего признакового пространства мы не знаем.
        В этом случае пользоваться bag-of-words или, например, CountVectorizer из sklearn, не 
        целесообразно (словарь придется пересчитывать при каждом появлении нового слова).

    w_ : dict {string: defaultdict(int)}
        Mapping тегов в словарь {<численный_индекс_признака>: <вес_в_модели>} (для каждого тега
        свой набор параметров модели). Словарь изменяемого размера со значением по умолчанию 0.
        
    w0_ : dict {string: float}
        Mapping тегов в веса w0 (смещения).
        
    train_frequency_dict_ : Counter
        Counter-объект {<численный_индекс_признака>: <число_вхождений>}. Выполняет подсчет числа
        вхождений признака на всей тренировочной выборке.
    
    loss_ : [double]
        Список значений функции потерь для последней используемой обучающей выборки.
    
    """
    def __init__(self, tags, strategy='ovr', learning_rate=0.1, lmbda=0.0002, gamma=0.1, 
                 store_frequency=True, tolerance=1e-16):
        self.vocab_ = {}
        self.w_ = {t: defaultdict(int) for t in tags}
        self.w0_ = {t: 0.0 for t in tags}
        self.train_frequency_dict_ = Counter()
        self.loss_ = []
        
        self.tags_ = set(tags)
        self.strategy_ = strategy
        self.learning_rate_ = learning_rate
        self.lmbda_ = lmbda
        self.gamma_ = gamma
        self.store_frequency_ = store_frequency
        self.tolerance_ = tolerance
        
    
    def fit(self, datasource, total=None, update_vocab=True, return_train_loss=False):
        """Fit/update the model by passing the datasource
        Обучение/дообучение модели одним проходом по источнику данных.

        Parameters
        ----------
        datasource : iterable
            Итерируемый объект как источник данных. Формат примера из обучающей выборки - строка
            с классифицируемым текстом и список тегов, разделенные знаком табуляции.

        total : int or None, default=None
            Информация о количестве строк в источнике данных для вывода прогресс-бара. В случае
            значения None, прогресс-бар не используется.

        update_vocab : bool, default=True
            Флаг режима добавления слов в словарь (признаковое пространство) во время обучения.
            
        return_train_loss : bool, default=False
            Флаг сохранения значений функции потерь для каждого примера из обучающей выборки.
            
        Returns
        -------
        self : object
            Возвращает объект класса
        """
        self.loss_ = [] 
        
        if total is not None:
            wrapped_source = tqdm_notebook(datasource, total=total, mininterval=1)
        else:
            wrapped_source = datasource
            
        for line in wrapped_source:
            
            input_pair = line.strip().split('\t')
            if len(input_pair) != 2:
                continue             
                
            word_sentence = input_pair[0].split(' ')
            sample_tags = set(input_pair[1].split(' '))
            
            # отбор только известных тегов
            sample_tags = sample_tags & self.tags_ 
            
            if len(sample_tags) == 0:
                continue

            # значение функции потерь для текущего примера
            sample_loss = 0

            # градиентный спуск для каждого тега
            for tag in self.tags_:
                
                # целевая переменная
                y = int(tag in sample_tags)

                # инициализируем z (линейная комбинация весов и признаков объекта) смещением
                z = self.w0_[tag]

                # чтобы не пробегать словарь на каждой строчке в процессе обучения, линейная 
                # комбинация весов и признаков z рассчитывается как сумма весов модели для 
                # каждого встреченного слова (если слово втречалось несколько раз, то и в 
                # итоговую сумму его вес войдет такое же число раз; для остальных слов из 
                # словаря вхождений не будет)
                for word in word_sentence:
                    
                    # обработка слова не из словаря
                    if word not in self.vocab_:
                        if update_vocab:
                            self.vocab_[word] = len(self.vocab_)
                        else:
                            continue

                    z += self.w_[tag][self.vocab_[word]]

                # вычисляем сигмоид (фактически, это вероятность наличия тега);
                # чтобы не столкнуться с overflow, избегаем вычисления экспоненты 
                # с очень большим по модулю положительным аргументом
                if z >= 0:
                    sigma = 1 / (1 + np.exp(-z))
                else:
                    sigma = 1 - 1 / (1 + np.exp(z))

                # обновляем значение функции потерь для текущего примера;
                # чтобы не получить потери точности, избегаем вычисления логарифма с
                # близким к 0 или 1 аргументом, используя порог tolerance
                if y == 1:
                    sample_loss += -1 * np.log(np.max([sigma, self.tolerance_]))
                else:
                    sample_loss += -1 * np.log(1 - np.min([1 - self.tolerance_, sigma]))

                # обновим параметры модели
                
                # вычисляем частную производную функции потерь по текущему весу;
                # учет xm будет реализовываться в цикле далее
                dHdw = (sigma - y)

                # делаем градиентный шаг и выполняем регуляризацию;
                # в целях увеличения производительности делаем допущение для регуляризации;
                # чтобы в каждой итерации обучения не выполнять регуляризацию всех параметров,
                # будем учитывать только присутствующие в текущем обучающем примере признаки;
                # естественно, каждый признак должен быть регуляризован только один раз, не
                # учитывая число его вхождений в обучающий пример;
                # будем выполнять регуляризацию во время первого появления признака
                
                regularized_words = set()

                for word in word_sentence: 
                    
                    if word not in self.vocab_:
                        continue

                    regularization = 0.0
                    
                    if self.lmbda_ > 0.0:
                        if self.vocab_[word] not in regularized_words:                    
                            regularized_words.add(self.vocab_[word])
                            w = self.w_[tag][self.vocab_[word]]    
                            regularization = self.lmbda_ * (2 * self.gamma_ * w + (1 - self.gamma_) * np.sign(w))

                    # корректировка веса значением антиградиента;
                    # явное указание 1.0 показывает, что мы не забыли множитель xm
                    self.w_[tag][self.vocab_[word]] -= self.learning_rate_ * (1.0 * dHdw + regularization)

                # смещение не регуляризируется
                self.w0_[tag] -= self.learning_rate_ * 1.0 * dHdw

            self.loss_.append(sample_loss)

            # обновим частотный словарь
            if self.store_frequency_ & update_vocab:
                self.train_frequency_dict_ += Counter([self.vocab_[word] for word in word_sentence])
                
        return self


    def filter_vocab(self, n=10000):
        """ Filtering vocabulary by the top-n words (for all classes)
        Отбор топ-n самых популярных слов в словаре (для всех тегов)

        Parameters
        ----------
        n : int, default=10000
            количество слов для отбора
            
        Returns
        -------
        self : object
            Возвращает объект класса
        """
        if not self.store_frequency_:
            print('can\'t filter vocabulary case no frequency data')
            return
        
        top_words = {k for (k, v) in self.train_frequency_dict_.most_common(n)}

        # обновим словарь
        self.vocab_ = {key: val for (key, val) in self.vocab_.items() if val in top_words}
        
        # обновим словари весов для тегов
        for tag in self.tags_:
            self.w_[tag] = {key: val for (key, val) in self.w_[tag].items() if key in top_words}
            
        return self


    def predict_proba(self, datasource, total=None):        
        """ 
        Предсказание тегов для текста
    
        Parameters
        ----------
        datasource : iterable
            Итерируемый объект как источник данных, содержащий строки для классификации
            
        total : int or None, default=None
            Информация о количестве строк в источнике данных для вывода прогресс-бара. В случае
            значения None, прогресс-бар не используется.
            
        Returns
        -------
        predicted : list of [(tag, probability), ...]
            Возвращает список, где элементом является список кортежей вида 
            (<тег>, <вероятность>)
        """
        predicted = []
        
        if total is not None:
            wrapped_source = tqdm_notebook(datasource, total=total, mininterval=1)
        else:
            wrapped_source = datasource
            
        for line in wrapped_source:
        
            sentence = line.strip().split(' ')
            line_predicted = dict()

            for tag in self.tags_:
                # расчитываем значение линейной комбинации весов и признаков объекта
                z = self.w0_[tag]

                for word in sentence:
                    if word not in self.vocab_:
                        continue
                    z += self.w_[tag][self.vocab_[word]] * 1.0

                # вычисляем вероятность наличия тега
                if z >= 0:
                    sigma = 1 / (1 + np.exp(-z))
                else:
                    sigma = 1 - 1 / (1 + np.exp(z))

                line_predicted[tag] = sigma
                
            predicted.append(line_predicted)
        
        return pd.DataFrame(predicted)
    
    
    def score(self, datasource, labels_datasource):
        """ Returns the mean Jaccard index on the given dataset and labels
        Возвращает метрику качества (средний коэффициент Жаккара) по выборке с 
        указанными метками
        
        Parameters
        ----------
        X : iterable
            Итерируемый объект как источник данных, содержащий строки для классификации
            
        total : int or None, default=None
            Информация о количестве строк в источнике данных для вывода прогресс-бара. В случае
            значения None, прогресс-бар не используется.
            
        Returns
        -------
        predicted : list of [(tag, probability), ...]
            Возвращает список, где элементом является список кортежей вида 
            (<тег>, <вероятность>)
        """
        pass

#### prepare to test

In [9]:
# загрузка настроек
with open('./settings.json', 'r') as settings_file:
    settings = json.load(settings_file)

print('keys in settings file:')
list(settings.keys())

keys in settings file:


['data_dir',
 'data_file',
 'top_tags_count',
 'top_tags_file',
 'filtered_tmp_file',
 'train_size',
 'train_file',
 'test_file',
 'test_labels_file']

In [10]:
top_tags_filepath = os.path.join(settings['data_dir'], settings['top_tags_file'])
train_filepath = os.path.join(settings['data_dir'], settings['train_file'])
test_filepath = os.path.join(settings['data_dir'], settings['test_file'])

In [11]:
# загрузка топ тегов
top_tags_dataframe = pd.read_csv(top_tags_filepath, header=None)
top_tags = top_tags_dataframe[0].tolist()

print('top-{} tags:'.format(settings['top_tags_count']))
top_tags

top-10 tags:


['javascript',
 'java',
 'c#',
 'php',
 'android',
 'jquery',
 'python',
 'html',
 'c++',
 'ios']

In [12]:
# размер тренировочного датасета
train_lines_count = int(subprocess.check_output(['wc', '-l', train_filepath]).split()[0])

In [13]:
##########

In [14]:
model_ss = OnlineLogisticRegression(top_tags, lmbda=0.0, store_frequency=False)

In [15]:
with open(train_filepath) as train_file:
    model_ss.fit(train_file, total=train_lines_count)

In [16]:
test_lines_count = int(subprocess.check_output(['wc', '-l', test_filepath]).split()[0])

with open(test_filepath) as test_file:
    predicted = model_ss.predict_proba(test_file, total=test_lines_count)

In [17]:
predicted.head()

,android,c#,c++,html,ios,java,javascript,jquery,php,python
0,3.461698e-11,6.277007e-07,5.107037e-10,2.767721e-07,1.254552e-13,1.645351e-13,0.000000e+00,0.0,1.000000,0.000000e+00
1,0.000000e+00,9.997549e-01,9.784114e-07,0.000000e+00,3.552714e-15,0.000000e+00,1.000000e+00,0.0,0.000000,0.000000e+00
2,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00
3,4.941168e-02,3.799459e-02,6.661338e-15,6.778254e-10,1.301085e-04,1.097661e-01,3.081024e-11,0.0,0.405418,8.858025e-12
4,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.000000,0.000000e+00


#### test model

In [ ]:
models_loss = dict()

In [ ]:
# классификатор с параметрами по умолчанию
# (регуляризация и подсчет частотного словаря активированы)
model_w_freq = OnlineLogisticRegression(top_tags)

In [ ]:
with open(train_filepath) as train_file:
    model_w_freq.fit(train_file, total=train_lines_count, return_train_loss=True)

In [ ]:
models_loss['lmbda=0.0002']= model_w_freq.loss_

In [ ]:
# отфильтруем словарь топ-10k словами и выполним еще одну итерацию обучения
model_w_freq.filter_vocab(10000)

with open(train_filepath) as train_file:
    model_w_freq.fit(train_file, total=train_lines_count, update_vocab=False, return_train_loss=True)

In [ ]:
models_loss['lmbda=0.0002, top-10k']= model_w_freq.loss_

In [ ]:
#  отфильтруем словарь топ-5k словами и выполним еще одну итерацию обучения
model_w_freq.filter_vocab(5000)

with open(train_filepath) as train_file:
    model_w_freq.fit(train_file, total=train_lines_count, update_vocab=False, return_train_loss=True)

In [ ]:
models_loss['lmbda=0.0002, top-5k']= model_w_freq.loss_

In [ ]:
# классификатор без регуляризации и частотного словаря
model_wo_freq = OnlineLogisticRegression(top_tags, lmbda=0.0, store_frequency=False)

with open(train_filepath) as train_file:
    model_wo_freq.fit(train_file, total=train_lines_count, return_train_loss=True)

In [ ]:
models_loss['lmbda=0.0, 1st pass']= model_wo_freq.loss_

In [ ]:
# второй проход по выборке
with open(train_filepath) as train_file:
    model_wo_freq.fit(train_file, total=train_lines_count, return_train_loss=True)

In [ ]:
models_loss['lmbda=0.0, 2nd pass']= model_wo_freq.loss_

#### results

In [ ]:
roll = 1000

plt.figure(figsize=(8,6))
for (key, val) in models_loss.items():
    plt.plot(pd.Series(val).rolling(roll).mean(), label=key)
plt.grid()
plt.legend()

pass